In [1]:
import cassandra;

In [2]:
from cassandra.cluster import Cluster

In [4]:
cluster = Cluster(['172.17.0.2'])
session = cluster.connect()

In [23]:
from cassandra import ConsistencyLevel
session.default_consistency_level = ConsistencyLevel.QUORUM;

results = session.execute("SELECT * from ex2.users")
for result in results:
    print(result.user_id, "Email:", result.email)

#Zmiana CL dla jednego query
from cassandra.query import SimpleStatement
stmt = SimpleStatement("SELECT * from ex2.users", consistency_level=ConsistencyLevel.ONE);
session.execute(stmt)

    

z_pythona Email: None
user2 Email: email1@domainl.com
user1 Email: email1@domainl.com
user3 Email: email1@domainl.com
user6 Email: email1@domainl.com


In [40]:
session.set_keyspace('ex2')
#lub session.execute('USE ex2')
#session.execute("INSERT INTO users (user_id) VALUES ('z_pythona')")

# to samo ze zmienna i w petli
#for a in range(10):
    #username = "zenek"+str(a)
    #email = "zenek@poland.pl"
    #session.execute("INSERT INTO users (user_id, email) VALUES (%s, %s)", [username, email])

    
#cql: update users set meta['klucz1'] = {'k1': [1,2,3]} WHERE user_id='user8';
results = session.execute("SELECT * from ex2.users where user_id='user8'")
for result in results:
    print(result.user_id, "Email:", result.email)
    print("META:", result.meta['klucz1']['k1'])
    
    
username = "marek"
email = "marek@poland.pl"
meta = {
    'klucz1': {
        'k1': [1,2,3]
    }
}
session.execute("INSERT INTO users (user_id, email, meta) VALUES (%s, %s, %s)", [username, email, meta])


user8 Email: None
META: [1, 2, 3]


In [74]:
from time import time

def on_ready(result):
    print("GOTOWY!", (time() - start) * 1000)

start = time()
futures = []
for i in range(0,100):
    username = "testuser_" + str(i)
    future = session.execute_async("INSERT INTO users (user_id) VALUES(%s)", [username])
    
    futures.append(future)
    future.add_callback(on_ready)
    
took = time() - start
print("Took", took*1000, "ms")

for future in futures:
    future.result()
print("All took:", (time() - start) * 1000)

Took 12.830734252929688 ms
GOTOWY! 16.433000564575195
GOTOWY! 16.604900360107422
GOTOWY! 16.782045364379883
GOTOWY! 16.91293716430664
GOTOWY! 17.040252685546875
GOTOWY! 17.209529876708984
GOTOWY! 17.342567443847656
GOTOWY! 17.477750778198242
GOTOWY! 17.626047134399414
GOTOWY! 17.94886589050293
GOTOWY! 18.117189407348633
GOTOWY! 18.250703811645508
GOTOWY! 18.42975616455078
GOTOWY! 18.552541732788086
GOTOWY! 18.702030181884766
GOTOWY! 18.83411407470703
GOTOWY! 18.9969539642334
GOTOWY! 19.155263900756836
GOTOWY! 19.27971839904785
GOTOWY! 19.400835037231445
GOTOWY! 19.52195167541504
GOTOWY! 19.643306732177734
GOTOWY! 19.765138626098633
GOTOWY! 19.886016845703125
GOTOWY! 20.014047622680664
GOTOWY! 20.151376724243164
GOTOWY! 20.274639129638672
GOTOWY! 20.470619201660156
GOTOWY! 20.64657211303711
GOTOWY! 20.940780639648438
GOTOWY! 21.65961265563965
GOTOWY! 21.840810775756836
GOTOWY! 22.682905197143555
GOTOWY! 23.17667007446289
GOTOWY! 23.354768753051758
GOTOWY! 23.532629013061523
GOTOWY! 23.9